https://blog.csdn.net/weixin_42608414/article/details/88614946

In [1]:
import pandas as pd
import chardet
import pandas as pd
import numpy as np
import jieba as jb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import gensim
from gensim import corpora
import pyLDAvis.gensim

C:\Anaconda3\lib\site-packages\funcy\colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
C:\Anaconda3\lib\site-packages\funcy\colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
C:\Anaconda3\lib\site-packages\funcy\colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
C:\Anaconda3\lib\site-packages\funcy\colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collectio

In [2]:
my_doc_list = pd.read_csv('./my_corpus_chinese.csv', names=['Label', 'Title', 'Text'])

my_doc_list.head()

C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Label,Title,Text
0,水果,一些 水果1,苹果 苹果 苹果 苹果 苹果 苹果 苹果 苹果 苹果 香蕉 香蕉 葡萄
1,水果,一些 水果2,苹果 苹果 苹果 苹果 苹果 苹果 苹果 苹果 香蕉 香蕉 葡萄 葡萄 轿车
2,车辆,一些 车辆1,葡萄 葡萄 葡萄 轿车 轿车 轿车 卡车 卡车 卡车 卡车 火车 火车 火车 火...
3,车辆,一些 车辆2,香蕉 香蕉 轿车 轿车 卡车 卡车 卡车 卡车 火车 火车 火车 火车 ...
4,混合,混合1,苹果 苹果 葡萄 葡萄 火车 火车 火车


In [3]:
#定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
 
#停用词列表
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  
 
#加载停用词
stopwords = stopwordslist("./my_stop_words_chinese.txt")


C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
import re

#删除除汉字以外的所有符号
my_doc_list['Cleaned_Text'] = my_doc_list['Text'].apply(remove_punctuation)
 
#分词，并过滤停用词
my_doc_list['Cut_Text'] = my_doc_list['Cleaned_Text'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
my_doc_list.head()

C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.370 seconds.
Prefix dict has been built successfully.


,Label,Title,Text,Cleaned_Text,Cut_Text
0,水果,一些 水果1,苹果 苹果 苹果 苹果 苹果 苹果 苹果 苹果 苹果 香蕉 香蕉 葡萄,苹果苹果苹果苹果苹果苹果苹果苹果苹果香蕉香蕉葡萄,苹果 苹果 苹果 苹果 苹果 苹果 苹果 苹果 苹果 香蕉 香蕉 葡萄
1,水果,一些 水果2,苹果 苹果 苹果 苹果 苹果 苹果 苹果 苹果 香蕉 香蕉 葡萄 葡萄 轿车,苹果苹果苹果苹果苹果苹果苹果苹果香蕉香蕉葡萄葡萄轿车,苹果 苹果 苹果 苹果 苹果 苹果 苹果 苹果 香蕉 香蕉 葡萄 葡萄 轿车
2,车辆,一些 车辆1,葡萄 葡萄 葡萄 轿车 轿车 轿车 卡车 卡车 卡车 卡车 火车 火车 火车 火...,葡萄葡萄葡萄轿车轿车轿车卡车卡车卡车卡车火车火车火车火车火车火车火车火车,葡萄 葡萄 葡萄 轿车 轿车 轿车 卡车 卡车 卡车 卡车 火车 火车 火车 火车 火车 火...
3,车辆,一些 车辆2,香蕉 香蕉 轿车 轿车 卡车 卡车 卡车 卡车 火车 火车 火车 火车 ...,香蕉香蕉轿车轿车卡车卡车卡车卡车火车火车火车火车火车火车火车,香蕉 香蕉 轿车 轿车 卡车 卡车 卡车 卡车 火车 火车 火车 火车 火车 火车 火车
4,混合,混合1,苹果 苹果 葡萄 葡萄 火车 火车 火车,苹果苹果葡萄葡萄火车火车火车,苹果 苹果 葡萄 葡萄 火车 火车 火车


In [5]:
no_features = 6
 
 
tfidf = TfidfVectorizer(max_features=no_features)
tfidf_features = tfidf.fit_transform( my_doc_list.Cut_Text)
tfidf_feature_names = tfidf.get_feature_names()

print("tfidf_features",tfidf_features)
print("tfidf_feature_names",tfidf_feature_names)

tfidf_features   (0, 3)	0.09086689624289442
  (0, 5)	0.21603302657828347
  (0, 2)	0.9721486196022756
  (1, 4)	0.11798994679890115
  (1, 3)	0.19851372585562138
  (1, 5)	0.2359798935978023
  (1, 2)	0.9439195743912092
  (2, 1)	0.789839261240196
  (2, 0)	0.4757550047131687
  (2, 4)	0.29618972296507345
  (2, 3)	0.24916413245848087
  (3, 1)	0.7815480615731121
  (3, 0)	0.538012399612908
  (3, 4)	0.22329944616374628
  (3, 5)	0.22329944616374628
  (4, 1)	0.7540001691930586
  (4, 3)	0.4228591416560611
  (4, 2)	0.5026667794620391
tfidf_feature_names ['卡车', '火车', '苹果', '葡萄', '轿车', '香蕉']


C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
cv = CountVectorizer(max_features=no_features)
cv_features = cv.fit_transform( my_doc_list.Cut_Text)
cv_feature_names = cv.get_feature_names()

print("cv_features",cv_features)
print("cv_feature_names",cv_feature_names)

cv_features   (0, 2)	9
  (0, 5)	2
  (0, 3)	1
  (1, 2)	8
  (1, 5)	2
  (1, 3)	2
  (1, 4)	1
  (2, 3)	3
  (2, 4)	3
  (2, 0)	4
  (2, 1)	8
  (3, 5)	2
  (3, 4)	2
  (3, 0)	4
  (3, 1)	7
  (4, 2)	2
  (4, 3)	2
  (4, 1)	3
cv_feature_names ['卡车', '火车', '苹果', '葡萄', '轿车', '香蕉']


C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
no_topics = 2
 
#NMF
nmf_tfidf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_features)

C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("主题 {} : {}".format(topic_idx,"|".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])))
 
no_top_words = 4
print('NMF-tfidf_features 主题')
display_topics(nmf_tfidf, tfidf_feature_names, no_top_words)

NMF-tfidf_features 主题
主题 0 : 火车|卡车|轿车|葡萄
主题 1 : 苹果|葡萄|香蕉|火车


C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
